In [15]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
elon = pd.read_csv('elon_data.csv',parse_dates=[0], infer_datetime_format=True)
elon = elon.drop(['Unnamed: 0'], axis=1)
elon = elon.dropna(axis=0)
bored_elon = pd.read_csv('bored_elon_data.csv',parse_dates=[0], infer_datetime_format=True)
bored_elon = bored_elon.drop(['Unnamed: 0'], axis=1)
bored_elon = elon.dropna(axis=0)

In [24]:
bored_elon.head()

,Tweet,Date,Retweets
0,@RanNatanzon @Tesla @Cortica This is completel...,Tue Mar 20 18:47:20 +0000 2018,195
1,Paid respects to Masada earlier today. Live fr...,Tue Mar 20 02:20:29 +0000 2018,844
2,Learning how to pour flaming absinthe over a t...,Mon Mar 19 18:09:26 +0000 2018,970
3,@IraEhrenpreis @Tesla Thanks for your support ...,Sun Mar 18 04:31:53 +0000 2018,157
4,@TheOnion Your cruel taunts cut me deep. Deep....,Thu Mar 15 18:46:45 +0000 2018,465


In [27]:
#remove punctuation from Tweet text
elon['Tweet'] = elon['Tweet'].str.replace('[^\w\s]','')
bored_elon['Tweet'] = bored_elon['Tweet'].str.replace('[^\w\s]','')

#add in label columns for data
elon['Label'] = "Elon"
bored_elon['Label'] = "BoredElon"

#join elon and bored_elon

In [28]:
bored_elon.head()

,Tweet,Date,Retweets,Label
0,RanNatanzon Tesla Cortica This is completely f...,Tue Mar 20 18:47:20 +0000 2018,195,BoredElon
1,Paid respects to Masada earlier today Live fre...,Tue Mar 20 02:20:29 +0000 2018,844,BoredElon
2,Learning how to pour flaming absinthe over a t...,Mon Mar 19 18:09:26 +0000 2018,970,BoredElon
3,IraEhrenpreis Tesla Thanks for your support ov...,Sun Mar 18 04:31:53 +0000 2018,157,BoredElon
4,TheOnion Your cruel taunts cut me deep Deep Bu...,Thu Mar 15 18:46:45 +0000 2018,465,BoredElon


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_tweets[“text”], all_tweets[“handle”], test_size=0.25, random_state=42, stratify=all_tweets[“handle”])

In [7]:
word_vectorizer = CountVectorizer(ngram_range=(1,2), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(elon['Tweet'])
frequencies = sum(sparse_matrix).toarray()[0]

pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])

AttributeError: 'map' object has no attribute 'lower'

In [49]:
data = pd.read_csv('data.csv')

In [50]:
data.head()

,Tweet,Date,Retweets
0,"All across this nation, we pray for our countr...",Wed Mar 14 00:34:44 +0000 2018,9018
1,It was my great honor to deliver a message at ...,Wed Mar 14 00:25:59 +0000 2018,7481
2,"If we don’t have a wall system, we’re not goin...",Tue Mar 13 22:23:26 +0000 2018,15458
3,California’s sanctuary policies are illegal an...,Tue Mar 13 15:27:18 +0000 2018,25657
4,“According to the Center for Immigration Studi...,Tue Mar 13 15:24:23 +0000 2018,16217


In [56]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data.dropna(subset=['Date'])

In [57]:
data.tail(50)

,Tweet,Date,Retweets
3108,The Great State of Michigan was just certified...,2016-11-29 11:15:49,20554
3109,.@CNN is so embarrassed by their total (100%)...,2016-11-29 03:03:59,23790
3112,@JoeBowman12: @jeffzeleny just another generic...,2016-11-29 02:15:54,7278
3113,@HighonHillcrest: @jeffzeleny what PROOF do u ...,2016-11-29 02:14:04,7508
3114,Just met with General Petraeus--was very impre...,2016-11-28 21:01:47,13176
3115,If Cuba is unwilling to make a better deal for...,2016-11-28 14:02:06,23395
3116,"Serious voter fraud in Virginia, New Hampshire...",2016-11-28 00:31:54,29588
3117,states instead of the 15 states that I visited...,2016-11-27 20:41:32,10262
3118,It would have been much easier for me to win t...,2016-11-27 20:34:18,11000
3119,In addition to winning the Electoral College i...,2016-11-27 20:30:43,50642


In [58]:
data.to_csv('data.csv', index=False)